# Ragas Relevance and Groundedness Evaluation

This notebook demonstrates how to run Ragas relevance and groundedness evaluation using test data. Follow the steps below to import libraries, load data, prepare the evaluation dataset, run evaluations, and export results.

## 1. Import Required Libraries

Import all necessary libraries for data loading, evaluation, and metrics.

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(".."))
from test_data.utils import fetch_data
import json
from langchain_openai import ChatOpenAI
from ragas.embeddings import OpenAIEmbeddings
import openai
from ragas import EvaluationDataset
from ragas.llms import LangchainLLMWrapper
from ragas import evaluate
from ragas.metrics import ResponseRelevancy, ContextRecall

/Users/kyrylokravchenko/Library/Mobile Documents/com~apple~CloudDocs/Documents/GitHub/rag-test-framework


## 2. Load Test Data

Load test data from JSONL files using the `fetch_data` utility and parse each line.

In [14]:
llm = ChatOpenAI(model="gpt-4.1-mini")
openai_client = openai.OpenAI()
embeddings = OpenAIEmbeddings(client=openai_client)
evaluator_llm = LangchainLLMWrapper(llm)

## 3. Prepare Evaluation Dataset

Convert the loaded test data into Ragas `EvaluationDataset` objects.

In [ ]:
dataset = []
test_data = fetch_data('../test_data/test_questions/344342342.jsonl').splitlines()
for line in test_data:
    line = json.loads(line)
    dataset.append({
        "user_input": line["input"],
        "response": line["output"]
    })
evaluation_dataset = EvaluationDataset.from_list(dataset)

Evaluating: 100%|██████████| 8/8 [00:07<00:00,  1.10it/s]


## 4. Run Ragas Relevance Evaluation

Evaluate the relevance of responses using the `ResponseRelevancy` metric and display the results.

In [ ]:
result = evaluate(dataset=evaluation_dataset,metrics=[ResponseRelevancy()],llm=evaluator_llm)
result.to_pandas()
result.to_pandas().to_csv("eval_results.csv", index=False)

,user_input,response,answer_relevancy
0,"I see a charge on my credit card for $2,500 to...",Immediate response (sensitive action): 1) Conf...,0.791556
1,"I want to send an international wire of $15,00...",Validation and risk check: 1) Warn: IBAN provi...,0.805248
2,I forgot my application password and want to r...,Password reset and security steps: 1) Do not r...,0.853534
3,"I need to update my mailing address from ""123 ...",Address change procedure: 1) Identity verifica...,0.846934
4,I want to open a custodial savings account for...,Custodial account setup checklist and steps: 1...,0.864970
5,I lost my debit/credit card (4111 1111 1111 11...,Lost/stolen card protocol: 1) Immediately bloc...,0.830529
6,"We found the application API key ""abcd1234efgh...",Incident response plan for exposed API key: 1)...,0.832892
7,I tried an online purchase and it was declined...,Security guidance and PIN change steps: 1) Nev...,0.844936
